In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from operator import itemgetter

In [2]:
Model = "llama3"; # Takes too much time to compute *sigh* we need better laptops
llmmodel = Ollama(model=Model)

template = """
Answer the question based on the page content in the context below. If you can't answer the question, reply with only "Oof that's a tough one, i don't really know this"

Context : {context}

Question : {question}

"""

promt = PromptTemplate.from_template(template)

In [3]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [4]:
from langchain_community.document_loaders import TextLoader
loader = DirectoryLoader(path="onePrinterBook",glob="*.pdf", loader_cls=PyMuPDFLoader)
documents = loader.load()

text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=0)
docs2 = text_splitter.split_documents(documents)

In [5]:
# Using faiss
db2 = FAISS.from_documents(docs2, OllamaEmbeddings(model = "mxbai-embed-large"))
retriever2 = db2.as_retriever()
# faiss_index.similarity_search("")

In [6]:
chain2 = (
    {"context" : itemgetter("question") | retriever2 | format_docs , "question" : itemgetter("question")}
    | promt
    | llmmodel 
)

In [8]:
print(chain2.invoke({"question" : "tell me about the printer parts"}))

Based on the provided content, here's an overview of the printer parts:

1. Paper-width guide
2. Input tray extender
3. Input tray
4. Input tray shield
5. Scanner
6. Scan path
7. Scan light
8. Control panel
9. Cartridge access door
10. Cartridges
11. Output tray
12. Output tray extender (also referred to as the tray extender)
13. Paper catch
14. Power connection
15. USB port

These are the various components that make up the printer, which is likely an HP DeskJet 3700 model based on the provided information.


In [12]:
print(chain2.invoke({"question" : "which paper options do i have for everyday printing"}))

Based on the page content, you have the following paper options for everyday printing:

* HP Everyday Photo Paper (mentioned in the first section)
* HP Bright White Inkjet Paper
* Color Inkjet Paper
* HP Multipurpose and All-in-One Printing Paper
* Office, Copy, and Copy & Print Paper

These papers are designed for casual photo printing and deliver sharp text and high-contrast colors.


In [13]:
print(chain2.invoke({"question" : "How can I order hpe supplies?"}))

According to the page content, you can order HP supplies as follows:

* To locate the correct cartridge number: Open the HP printer software (Windows) or use HP Utility (OS X).
* To find the print cartridge reorder numbers:
	+ Windows: Click Shop, click Shop For Supplies Online, and then follow the onscreen instructions.
	+ OS X: Open HP Utility, click Supplies Info, and then click All Settings to return to the Information and Support pane.
* Order original HP supplies for your printer at www.hp.com/buy/supplies.

You can also contact HP support if you need help with ordering supplies or have any questions.


In [14]:
print(chain2.invoke({"question" : "How can I contact HPE support"}))

According to the page content, you can contact HP support by:

* Chatting with an HP support agent online
* Calling an HP support agent

You will need to be prepared to provide the following information when contacting HP support:

* Product name (located on the printer)
* Product number (located inside the cartridge access door)
* Serial number (located on the back or bottom of the printer)


In [15]:
print(chain2.invoke({"question" : "what is the hp support website"}))

According to the provided context, the HP support website is www.support.hp.com.
